# Predictive Maintenance

#### This project uses the power of machine learning for early detection of equipment failure or equipment condition in order to </br></br> preemptively trigger a maintenance alert to avoid machine failure which could have lead to serious consequences.

</br> 

* In this project, we will be analyzing airplane engine data but this can be applied to other machinery as well. We will try to predict the remaining useful life of the aircraft engine.</br></br>

* In summary, the scenario employs simulated aircraft data from 21 sensors to anticipate when an aircraft engine would fail in the future, allowing for proactive maintenance planning.


In [1]:
#importing the libraries

import pandas as pd
import numpy as np
import keras
import os

from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation

## 1. Data Preparation

We will download the data from azure sample data for machine learning. We will downloading three files :-

* <b>PM_train.txt</b> - The training data consists of multiple multivariate time series with "cycle" as the time unit, together with 21 sensor readings for each cycle. Each time series can be assumed as being generated from a different engine of the same type. When a predefined threshold is reached, then the engine is considered unsafe for further operation. </br><b>In other words, the last cycle in each time series can be considered as the failure point of the corresponding engine.</b></br></br>

* <b>PM_test.txt</b> - The testing data has the same data schema as the training data. The only difference is that the data does not indicate when the failure occurs (in other words, the last time period does NOT represent the failure point). It is not shown how many more cycles this engine can last before it fails.</br></br>

* <b>PM_truth.txt</b> - The ground truth data ("RUL_FD001.txt") provides the number of remaining working cycles for the engines in the testing data.

### 1.1 Data Loading

In [2]:
# Loading the data into dataframes

train_df = pd.read_csv('PM_train.txt', sep = ' ', header = None)
test_df = pd.read_csv('PM_test.txt', sep = ' ', header = None)
truth_df = pd.read_csv('PM_truth.txt', sep = ' ', header = None)

#Dropping unnecessary columns
train_df.drop(train_df.columns[[26,27]], axis = 1, inplace = True)
test_df.drop(test_df.columns[[26,27]], axis = 1, inplace = True)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

#Renaming Columns
columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

train_df.columns = columns
test_df.columns = columns

#Sorting Dataframe
train_df = train_df.sort_values(['id', 'cycle'])

### 1.2 Data preprocessing

As we assume that last observation in the train dataframe is the failure point, we need to generate labels. We will calculate the Remaining Useful Life (RUL) column for each cycle in the data.

In [3]:
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']

train_df = train_df.merge(rul, on = ['id'], how = 'left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis = 1, inplace = True)
#train_df.head()

Let's create a new lable for time to failure. We will generate two labels:-
* label_1 - Indicating engine will fail within a month(30 days)
* label_2 - Indicating healthy, RUL <= 30, RUL <= 15

In [4]:
#label_1
train_df['label_1'] = np.where(train_df['RUL'] <= 30, 1, 0)

#label_2
train_df['label_2'] = np.where(train_df['RUL'] <= 30, 1, 0)
train_df.loc[train_df['RUL'] <= 15, 'label_2'] = 2
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,RUL,label_1,label_2
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,0,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,0,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,0,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,0,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,0,0


We need to normalize columns as we need all the values to use common scale without distorting the differences in the ranges of values or losing information.

In [5]:
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label_1','label_2'])
min_max_scaler = MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
#train_df.head()

In [6]:
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
#test_df.head()

Now we will need to generate labels for test data using the truth dataset

In [7]:
temp = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
temp.columns = ['id', 'max']
truth_df.columns = ['add']
truth_df['id'] = 1 + truth_df.index
truth_df['max'] = temp['max'] + truth_df['add']
truth_df.drop('add', axis = 1, inplace = True)

#Creating Remaining useful Life(RUL) from test data
test_df = test_df.merge(truth_df, on=['id'], how = 'left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis = 1, inplace = True)

test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm,RUL
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000,142.0
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277,141.0
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554,140.0
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831,139.0
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108,138.0


Now we need to create the same labels like training data:- 
* label_1 - Indicating engine will fail within a month(30 days)
* label_2 - Indicating healthy, RUL <= 30, RUL <= 15

In [8]:
#label_1
test_df['label_1'] = np.where(test_df['RUL'] <= 30, 1, 0)

#label_2
test_df['label_2'] = np.where(test_df['RUL'] <= 30, 1, 0)
test_df.loc[test_df['RUL'] <= 15, 'label_2'] = 2
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label_1,label_2
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000,142.0,0,0
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277,141.0,0,0
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554,140.0,0,0
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831,139.0,0,0
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108,138.0,0,0


## 2. Modelling

There are various options in Recurrent Neural Networks(RNN), but using LSTM(Long Short Term Memory) since there can be lags of unknown duration between important events in a time series. LSTMs were developed to deal with the vanishing gradient problem that can be encountered when training traditional RNNs. 

LSTM is used in time-series domains, and one of the parameter is the sequence length. We need to define cycles for rolling features. 

We allow the LSTMs to derive abstract features from the sequence of sensor data within the window using deep learning. Patterns within these sensor readings are expected to be automatically encoded by the LSTM.

Another key benefit of LSTMs is their capacity to recall long-term sequences (window sizes), which is difficult to achieve with standard feature engineering. Due to the smoothing over such a lengthy time, computing rolling averages with a window size of 50 cycles may result in information loss. LSTMs can work with wider windows and use all of the data in the window as input.

We will use sequel length of 50 cycles to predict the probability of engine failure within 30 days.

In [9]:
sequence_length = 50

We need a  function to reshape features into an array of 3 dimensions(samples, time steps, features)

In [10]:
'''No padding is used, and only sequences that fit the window-length are examined. This implies that for testing purposes, 
those that are shorter than the window-length must be removed. Pad sequences so that we may utilise shorter ones as an option'''

def sequence(id_df, seq_length, seq_cols):
    array = id_df[seq_cols].values
    num_elem = array.shape[0]
    
    for start, stop in zip(range(0, num_elem - seq_length), range(seq_length, num_elem)):
        yield array[start:stop, :]

The sequences are built from the features(sensor and settings) values across the time steps(cycles) within each engine.

In [11]:
# pick the feature columns 
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
key_cols = ['id', 'cycle']
label_cols = ['label_1', 'label_2', 'RUL']

input_features = test_df.columns.values.tolist()
sensor_cols = [x for x in input_features if x not in set(key_cols)]
sensor_cols = [x for x in sensor_cols if x not in set(label_cols)]
sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

# The time is sequenced along
# This may be a silly way to get these column names, but it's relatively clear
sequence_cols.extend(sensor_cols)

print(sequence_cols)

['setting1', 'setting2', 'setting3', 'cycle_norm', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']


In [12]:
# Generator for the sequences

seq_gen = (list(sequence(train_df[train_df['id'] == id], sequence_length,
                        sequence_cols)) for id in train_df['id'].unique())

seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(15631, 50, 25)

Now we need to create a function to label these sequences

In [13]:
# function to generate labels

def gen_labels(id_df, seq_length, label):
    array = id_df[label].values
    num_elements = array.shape[0]
    return array[seq_length:num_elements, :]

As we are focusing on predicting failure within the 30 days, that is, label_1.

In [14]:
#generate labels

label_gen = [gen_labels(train_df[train_df['id'] == id], sequence_length,
                       ['label_1']) for id in train_df['id'].unique()]

label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(15631, 1)

### LSTM Model

The network architecture must be determined before a Neural Net can be built. In this case, we'll create a two-layer network with dropout. The initial LSTM layer has 100 units, one for each input sequence, and is followed by a 50-unit LSTM layer. Dropout will be applied to each LSTM layer to prevent overfitting. The sigmoid activation used in the last dense output layer corresponds to the binary classification criterion.

In [15]:
# build the network
# Feature weights
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

# LSTM model
model = Sequential()

# The first layer
model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))

# Plus a 20% dropout rate
model.add(Dropout(0.2))

# The second layer
model.add(LSTM(
          units=50,
          return_sequences=False))

# Plus a 20% dropout rate
model.add(Dropout(0.2))

# Dense sigmoid layer
model.add(Dense(units=nb_out, activation='sigmoid'))

# With adam optimizer and a binary crossentropy loss. We will opimize for model accuracy.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Verify the architecture 
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 100)           50400     
                                                                 
 dropout (Dropout)           (None, 50, 100)           0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                30200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 80,651
Trainable params: 80,651
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.callbacks import EarlyStopping

In [17]:
%%time
# fit the network
model.fit(seq_array, # Training features
          label_array, # Training labels
          epochs=10,   # We'll stop after 10 epochs
          batch_size=200, # 
          validation_split=0.10, # Use 10% of data to evaluate the loss. (val_loss)
          verbose=1, #
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', # Monitor the validation loss
                                                     min_delta=0,    # until it doesn't change (or gets worse)
                                                     patience=5,  # patience > 1 so it continutes if it is not consistently improving
                                                     verbose=0, 
                                                     mode='auto')]) 


Epoch 1/10
71/71 [==============================] - 10s 104ms/step - loss: 0.2657 - accuracy: 0.8897 - val_loss: 0.1237 - val_accuracy: 0.9444
Epoch 2/10
71/71 [==============================] - 7s 96ms/step - loss: 0.1025 - accuracy: 0.9594 - val_loss: 0.0945 - val_accuracy: 0.9565
Epoch 3/10
71/71 [==============================] - 7s 100ms/step - loss: 0.0870 - accuracy: 0.9633 - val_loss: 0.0558 - val_accuracy: 0.9904
Epoch 4/10
71/71 [==============================] - 7s 100ms/step - loss: 0.0794 - accuracy: 0.9676 - val_loss: 0.0447 - val_accuracy: 0.9872
Epoch 5/10
71/71 [==============================] - 7s 102ms/step - loss: 0.0676 - accuracy: 0.9730 - val_loss: 0.0517 - val_accuracy: 0.9770
Epoch 6/10
71/71 [==============================] - 7s 100ms/step - loss: 0.0628 - accuracy: 0.9735 - val_loss: 0.0537 - val_accuracy: 0.9706
Epoch 7/10
71/71 [==============================] - 7s 101ms/step - loss: 0.0676 - accuracy: 0.9716 - val_loss: 0.0551 - val_accuracy: 0.9725
Epoch 

In [18]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Training Accurracy: {}'.format(scores[1]))

79/79 [==============================] - 3s 39ms/step - loss: 0.0768 - accuracy: 0.9648
Training Accurracy: 0.9648135304450989


In [19]:
test_df.shape

(11939, 30)

### Model Testing

In [26]:
test = test_df.drop(label_cols, axis = 1)

In [39]:
def run(test): 
    # Create the sequences
    sequence_length = 50
    sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
    key_cols = ['id', 'cycle']

    # Feature engineering
    input_features = test.columns.values.tolist()
    sensor_cols = [x for x in input_features if x not in set(key_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    sequence_cols.extend(sensor_cols)
    
    seq_array = [test[test['id']==id][sequence_cols].values[-sequence_length:] 
                 for id in test['id'].unique() if len(test[test['id']==id]) >= sequence_length]

    seq_array = np.asarray(seq_array).astype(np.float32)
    try:
        prediction = model.predict(seq_array)
        #print(prediction)
        pred = prediction.tolist()
        return(pred)
    except Exception as e:
        return(str(e))

In [167]:
predictions = []
for x in range(0,100):
    try:
        tst_df = test.loc[test['id'] == x]
        ans=run(tst_df)
        predictions.append(ans[0][0])
    except:
        pass

df = pd.DataFrame(predictions, columns = ['pred'])
df['machine_number'] = df.index + 1


In [168]:
sorted_df = df[~df['pred'].str.contains('U', na = False)]
sorted_df.sort_values(by='pred', ascending=False)


,pred,machine_number
68,0.998481,69
76,0.998239,77
31,0.998216,32
34,0.998121,35
42,0.997457,43
...,...,...
7,0.000389,8
65,0.000369,66
23,0.0003,24
78,0.000213,79


In [170]:
machine_number = int(input('Enter Machine Number : '))
try:
    num = round(df[df['machine_number'] == machine_number].iloc[0][0]*100,5)
    print('Probability of engine failure in the next 30 days : ', 
     str(num), '%')
except:
    print('\nInformation insufficient')

Enter Machine Number : 19
Probability of engine failure in the next 30 days :  67.99344 %
